In [1]:

import numpy as np
import torch
import sys
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from collections import Counter
sys.path.append('..')

# DeepCASE Imports
from deepcase.preprocessing   import Preprocessor
from deepcase.context_builder import ContextBuilder
from deepcase.interpreter     import Interpreter

import matplotlib.pyplot as plt
import pandas as pd
import random

DEBUG = True
context_length = 50

if DEBUG:
    config = {
    'learning_rates': [0.001],
    'eps':            [0.2],        # Epsilon value to use for DBSCAN clustering, in paper this was 0.1 
    'threshold':      [0.2]
    }
    Epochs = 2
else:
    config = {
        'learning_rates': [0.001,0.0001,0.00001],
        'eps':            [0.3,0.2,0.15,0.1],        # Epsilon value to use for DBSCAN clustering, in paper this was 0.1 
        'threshold':      [0.2,0.15,0.1,0.05,0.01]         # Confidence threshold used for determining if attention from the ContextBuilder can be used, in paper this was 0.2
        }
    Epochs = 50

########################################################################
#                             Loading data                             #
########################################################################

# Create preprocessor
preprocessor = Preprocessor(
    length  = context_length,    # 10 events in context
    timeout = 86400, # Ignore events older than 1 day (60*60*24 = 86400 seconds)
)
    

/home/tengfei/anaconda3/envs/pytorch-1.12.1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# prepare data-drop duplicated

In [ ]:
'''
for find origin index from data file, through 'indexIntraining' in Interpreter.predict()
'''
# Load data from file
fox_alerts_ori = '../Datasets/AIT-ADS/alerts_csv_labels/fox_alerts.txt'
harrison_alerts_ori = '../Datasets/AIT-ADS/alerts_csv_labels/harrison_alerts.txt'
russellmitchell_alerts_ori = '../Datasets/AIT-ADS/alerts_csv_labels/russellmitchell_alerts.txt'
santos_alerts_ori = '../Datasets/AIT-ADS/alerts_csv_labels/santos_alerts.txt'
shaw_alerts_ori = '../Datasets/AIT-ADS/alerts_csv_labels/shaw_alerts.txt'
wardbeck_alerts_ori = '../Datasets/AIT-ADS/alerts_csv_labels/wardbeck_alerts.txt'
wheeler_alerts_ori = '../Datasets/AIT-ADS/alerts_csv_labels/wheeler_alerts.txt'
wilson_alerts_ori = '../Datasets/AIT-ADS/alerts_csv_labels/wilson_alerts.txt'

# find the index of duplicated records.
filelist=[fox_alerts_ori,harrison_alerts_ori,russellmitchell_alerts_ori,santos_alerts_ori,shaw_alerts_ori,wardbeck_alerts_ori,wheeler_alerts_ori,wilson_alerts_ori]
for i in range(len(filelist)):
    data_temp = pd.read_csv(filelist[i],sep=',')
    print(filelist[i])
    print(len(data_temp))
    dup_bool=data_temp.duplicated(subset=['time','ip','short']) 
    dup_indexs=np.where(dup_bool==False)[0]    
    print(len(dup_indexs))

In [ ]:
#drop duplicated records and store in new files.
filelist=[fox_alerts,harrison_alerts,russellmitchell_alerts,santos_alerts,shaw_alerts,wardbeck_alerts,wheeler_alerts,wilson_alerts]
for i in range(len(filelist)):
    data_temp = pd.read_csv(filelist[i],sep=',')
    print(filelist[i])    
    data_temp = data_temp.drop_duplicates(subset=['time','ip','short']) 
    # Save DataFrame to a text file
    new_name=filelist[i].split('.txt')[0]+'_uniq'+'.txt'
    data_temp.to_csv(new_name, index=False) 
    print("DataFrame saved to "+new_name)

# DeepCASE

In [2]:
# Load data from file
fox_alerts = '../Datasets/AIT-ADS/alerts_csv_labels/fox_alerts_uniq.txt'
harrison_alerts = '../Datasets/AIT-ADS/alerts_csv_labels/harrison_alerts_uniq.txt'
russellmitchell_alerts = '../Datasets/AIT-ADS/alerts_csv_labels/russellmitchell_alerts_uniq.txt'
santos_alerts = '../Datasets/AIT-ADS/alerts_csv_labels/santos_alerts_uniq.txt'
shaw_alerts = '../Datasets/AIT-ADS/alerts_csv_labels/shaw_alerts_uniq.txt'
wardbeck_alerts = '../Datasets/AIT-ADS/alerts_csv_labels/wardbeck_alerts_uniq.txt'
wheeler_alerts = '../Datasets/AIT-ADS/alerts_csv_labels/wheeler_alerts_uniq.txt'
wilson_alerts = '../Datasets/AIT-ADS/alerts_csv_labels/wilson_alerts_uniq.txt'

data =preprocessor.read_csv_files(fox_alerts,harrison_alerts,russellmitchell_alerts,santos_alerts,shaw_alerts,wardbeck_alerts,wheeler_alerts,wilson_alerts)
data.reset_index(drop=True, inplace=True)
data = data.rename(columns = {"time":"timestamp","ip":"machine","short":"event","time_label":"label"})
context, events,  mapping, mapping_label, labels=preprocessor.sequence(data,labels=None,verbose=False)


In [4]:
print(f'the total number of samples: {len(labels)}')
print(f'the number of false positive: {sum(labels==0)}')
print(f'the number of analomal samples: {sum(labels==1)} ')

the total number of samples: 380588
the number of false positive: 357548
the number of analomal samples: 23040 


In [3]:
# In case no labels are provided, set labels to -1
# IMPORTANT: If no labels are provided, make sure to manually set the labels
# before calling the interpreter.score_clusters method. Otherwise, this will
# raise an exception, because scores == NO_SCORE cannot be computed.
if labels is None:
    labels = np.full(events.shape[0], -1, dtype=int)    
    
''' using pandas.dataframe to save index for all data need to split through using train_test_split() ; 
context, labels, and events need to split at the same time for corresponse.(TF)'''
frames=[data,pd.DataFrame(context.detach().numpy()) ]  # 
X = pd.concat(frames, axis=1)  
y = pd.Series(labels)
X_train, X_test, labels_train_binary,labels_test_binary = train_test_split(X,y,test_size=0.4,random_state=35)

context_train=torch.tensor( X_train.loc[:,0:(context_length-1)].to_numpy() )
events_train=torch.tensor( X_train['event'].to_numpy()  )
context_test=torch.tensor( X_test.loc[:,0:(context_length-1)].to_numpy()   )
events_test=torch.tensor( X_test['event'].to_numpy()   )

labels_train_binary = torch.tensor(labels_train_binary.to_numpy() )
labels_test_binary = torch.tensor(labels_test_binary.to_numpy() )
    
# Cast to cuda if available
if torch.cuda.is_available():
    events_train  = events_train.to('cuda')
    events_test  = events_test.to('cuda')
    context_train = context_train.to('cuda')
    context_test = context_test.to('cuda')


In [4]:
########################################################################
#                         Using ContextBuilder                         #
########################################################################

vocab_size = len(np.unique(list(mapping)))

context_builder = ContextBuilder(
    input_size    = vocab_size,   # Number of input features to expect
    output_size   = vocab_size,   # Same as input size
    hidden_size   = 128,   # Number of nodes in hidden layer, in paper we set this to 128
    max_length    = context_length,    # Length of the context, should be same as context in Preprocessor
)

# Cast to cuda if available
if torch.cuda.is_available():
    context_builder = context_builder.to('cuda')

# Train the ContextBuilder
context_builder.fit(
    X             = context_train,               # Context to train with
    y             = events_train.reshape(-1, 1), # Events to train with, note that these should be of shape=(n_events, 1)
    labels        = labels_train_binary,
    epochs        = Epochs,                          # Number of epochs to train with
    batch_size    = 128,                         # Number of samples in each training batch, in paper this was 128
    learning_rate = 0.001,                        # Learning rate to train with, in paper this was 0.01
    verbose       = True,                        # If True, prints progress
)


[Epoch 2/2 loss=0.013399]: 100%|████████████████████████████████████████████████████| 1784/1784 [00:40<00:00, 44.33it/s]


ContextBuilder(
  (embedding): Embedding(76, 128)
  (embedding_one_hot): EmbeddingOneHot()
  (encoder): Encoder(
    (embedding): EmbeddingOneHot()
    (recurrent): GRU(76, 128, batch_first=True)
  )
  (decoder_attention): DecoderAttention(
    (embedding): Embedding(76, 128)
    (recurrent): GRU(128, 128, batch_first=True)
    (attn): Linear(in_features=128, out_features=50, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (decoder_event): DecoderEventTransformer(
    (em): Sequential(
      (0): Linear(in_features=76, out_features=128, bias=True)
      (1): ReLU()
    )
    (transformer_layer): TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
      )
      (linear1): Linear(in_features=128, out_features=2048, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (linear2): Linear(in_features=2048, out_features=128, bias=True)
      (norm1): LayerNorm

In [5]:
########################################################################
#                          Using Interpreter                           #
########################################################################

interpreter = Interpreter(
    context_builder = context_builder, # ContextBuilder used to fit data
    features        = 100,             # Number of input features to expect, should be same as ContextBuilder
    eps             = 0.1,             # Epsilon value to use for DBSCAN clustering, in paper this was 0.1  # try 0.2 Jia
    min_samples     = 5,               # Minimum number of samples to use for DBSCAN clustering, in paper this was 5
    threshold       = 0.2,             # Confidence threshold used for determining if attention from the ContextBuilder can be used, in paper this was 0.2
)

# Cluster samples with the interpreter
clusters = interpreter.cluster(
    X          = context_train,               # Context to train with
    y          = events_train.reshape(-1, 1), # Events to train with, note that these should be of shape=(n_events, 1)
    iterations = 100,                         # Number of iterations to use for attention query, in paper this was 100
    batch_size = 1024,                        # Batch size to use for attention query, used to limit CUDA memory usage
    verbose    = True,                        # If True, prints progress
)

Clustering: 100%|███████████████████████████████████████████████████████████████████████| 20/20 [02:23<00:00,  7.18s/it]


In [6]:
########################################################################
#                             Manual mode                              #
########################################################################

# Compute scores for each cluster based on individual labels per sequence
scores = interpreter.score_clusters(
    scores   = labels_train_binary, # Labels used to compute score (either as loaded by Preprocessor, or put your own labels here)
    strategy = "min",        # Strategy to use for scoring (one of "max", "min", "avg")
    NO_SCORE = -1,           # Any sequence with this score will be ignored in the strategy.
                                # If assigned a cluster, the sequence will inherit the cluster score.
                                # If the sequence is not present in a cluster, it will receive a score of NO_SCORE.
)

# Assign scores to clusters in interpreter
# Note that all sequences should be given a score and each sequence in the
# same cluster should have the same score.
interpreter.score(
    scores  = scores, # Scores to assign to sequences
    verbose = True,   # If True, prints progress
)


Scoring: 100%|██████████████████████████████████████████████████████████████████████████| 17/17 [00:05<00:00,  2.88it/s]


In [ ]:
########################################################################
#                        (Semi-)Automatic mode                         #
########################################################################

#Compute predicted scores
prediction, indexIntraining = interpreter.predict(
    X          = context_test,               # Context to predict
    y          = events_test.reshape(-1, 1), # Events to predict, note that these should be of shape=(n_events, 1)
    iterations = 100,                        # Number of iterations to use for attention query, in paper this was 100
    batch_size = 1024,                       # Batch size to use for attention query, used to limit CUDA memory usage
    verbose    = True,                       # If True, prints progress
)

#Compute the accuracy
mask_p_n = np.where((prediction ==0) | (prediction ==1))[0]
result_predicted = prediction[mask_p_n]
labels_test_bin = labels_test_binary[mask_p_n]
with open("output.txt", "a") as f:
    print(classification_report(labels_test_bin, result_predicted,digits=4,zero_division=0.0),file=f)
    print(f'the total number of samples: {prediction.shape[0]}', file=f)
    print(f'the number of predicted  samples: {sum(prediction ==1)+sum(prediction ==0)}',file=f)
    print(f'the percentage of predicted samples:{(sum(prediction ==1)+sum(prediction ==0))/prediction.shape[0]}',file=f)
    print(f'the number of samples that can not predicted automatically:{prediction.shape[0]-sum(prediction ==1)-sum(prediction ==0)}',file=f)
    print(f'the number of predicted positive samples: {sum(prediction ==1)}',file=f)
    print(f'the number of predicted negative samples:{sum(prediction ==0)}',file=f)
    print(f'the experiment date is:{time.ctime()}',file=f)
    print(50 *'*',file=f)


In [14]:
print(classification_report(labels_test_bin, result_predicted,digits=4,zero_division=0.0))
print(f'the total number of samples: {prediction.shape[0]}')
print(f'the number of predicted  samples: {sum(prediction ==1)+sum(prediction ==0)}')
print(f'the percentage of predicted samples:{(sum(prediction ==1)+sum(prediction ==0))/prediction.shape[0]}')
print(f'the number of samples that can not predicted automatically:{prediction.shape[0]-sum(prediction ==1)-sum(prediction ==0)}')
print(f'the number of predicted positive samples: {sum(prediction ==1)}')
print(f'the number of predicted false positive samples:{sum(prediction ==0)}')
print(f'the experiment date is:{time.ctime()}')
print(50 *'*')

              precision    recall  f1-score   support

           0     0.9830    1.0000    0.9914    131256
           1     1.0000    0.2543    0.4054      3052

    accuracy                         0.9831    134308
   macro avg     0.9915    0.6271    0.6984    134308
weighted avg     0.9833    0.9831    0.9781    134308

the total number of samples: 152236
the number of predicted  samples: 134308
the percentage of predicted samples:0.8822354764970178
the number of samples that can not predicted automatically:17928
the number of predicted positive samples: 776
the number of predicted false positive samples:133532
the experiment date is:Wed Dec  4 16:12:56 2024
**************************************************


# LLM (Ollama 3.1)

## prepare the input data to LLM

In [21]:
X_train_sort= X_train.sort_values(by='timestamp')

def get_example_context(index_in_prediction, context_number):
    example_record=X_train.iloc[indexIntraining[index_in_prediction]]
    X_train_sort_machine=X_train_sort.loc[(X_train_sort['machine'] == example_record['machine']) & (X_train_sort['timestamp'] <= example_record['timestamp'])]                           
    return X_train_sort_machine.iloc[context_number:,:7]

In [45]:
example_context_number=-11  # context( =10) + event = 11

# example_normal_predic_index=np.where(prediction==1)[0][0]   # the example of the first attack record
example_normal_predic_index=np.where(prediction==0)[0][0]  # the example of the first false positive record

predict_event=X_test.iloc[example_normal_predic_index, :7] # the event record to predict
    
''' transfer the flag of 'event' and 'label' back to words for Ollama to explain'''
for k,v in mapping.items():
    X_train_sort.loc[X_train_sort['event']==k,'event']=v
    if predict_event.loc['event']==k:
        predict_event.loc['event']=v
    
for k,v in mapping_label.items():    
    X_train_sort.loc[X_train_sort['label']==k,'label']=v
    if predict_event.loc['label']==k:
        predict_event.loc['label']=v



In [46]:

predict_withcontext=get_example_context(example_normal_predic_index,example_context_number)
predict_withcontext=predict_withcontext.append(predict_event)  # pd.DataFrame(.cpu())])

example_number=5
examples_record=[]
for ind in range(example_number):
    example_record=X_train.iloc[random.randint(0, X_train.shape[0] +example_context_number-1)]
    X_train_sort_machine=X_train_sort.loc[(X_train_sort['machine'] == example_record['machine']) & (X_train_sort['timestamp'] <= example_record['timestamp'])]                           
    examples_record.append( X_train_sort_machine.iloc[example_context_number:,:7] )  # the last one event should be the target event.



/tmp/ipykernel_1769042/973161820.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  predict_withcontext=predict_withcontext.append(predict_event)  # pd.DataFrame(.cpu())])


In [24]:
def summizeCorpus(df):
    # summarize for the events at the same ip address
    # return: list results
    # Initialize variables
    start_time = df.iloc[0]['timestamp']
    pre_event  = df.iloc[0]['name']
    pre_host   = df.iloc[0]['host']
    pre_short  = df.iloc[0]['event']
    pre_label  = df.iloc[0]['label']
    count = 1
    
    results = []
    
    for i in range(1, len(df)):
        row = df.iloc[i]
        if (row['name'] == pre_event) and (row['host'] == pre_host) and (row['event'] == pre_short) and (row['label'] == pre_label):
            count += 1
        else:
            end_time = df.iloc[i-1]['timestamp']
            results.append(f"{start_time}-{end_time}  {pre_event}  {pre_host} {pre_short}  {pre_label}  counts:{count}")
            
            # reset start time
            start_time = row['timestamp']
            pre_event  = row['name']
            pre_host   = row['host']
            #pre_short  = row['short']
            pre_label  = row['label']
            count = 1
    end_time = df.iloc[-1]['timestamp']
    results.append(f"{start_time}-{end_time}  {pre_event}  {pre_host}  {pre_label}  counts:{count}")
    
    return results

def groupby_ip_event(df,abbr1 = 'machine', abbr2='event'):
    
    grouped_df = df.groupby([abbr1,abbr2])
    
    return grouped_df


In [ ]:
data_group_ip_event = groupby_ip_event(X_train_sort)

#item 1: data to train for getting the rules in llm.
X_train_sort_IP=pd.concat(subgroup for _, subgroup in data_group_ip_event)  # data for training in llm
X_train_sort_IP_sum=summizeCorpus(X_train_sort_IP)

''' the train set for llm should be the concluded/processed context, e.g. from contextbuilder. It should not be the event sequence following time.
item2: data for examples in prediction in llm.  examples_record  ( not limited to one IP)
item3: context data for prediction event in prediction in llm.  predict_withcontext
'''

In [40]:
def var4prompt(df_list, n=4):
    """ Assign value to variables in the prompt
        Input: list contains 5 df, each df has consecutive 11 events
               n is the length of df_list, the first 4 frames are samples, the 5th is sample to predict
        Output: Dict variables, the keys are:
                                example_1
                                event_1
                                label_1
                                ...
                                example_4
                                event_4
                                label_4
                                
                                task_precedings
                                task_event
                                  """
    context = {}
    for i in range(n):
        
        df = df_list[i]
        array_df = df.to_numpy() # convert df to a array
        
        context[f"example_{i+1}"] = array_df[0:10,0:5]
        context[f"event_{i+1}"]   = array_df[10][0:5]
        context[f"label_{i+1}"]   = array_df[10][5]        
    
    return context    

## LLM detection and explanation

In [31]:
''' Ollma package, implement using text completion'''
from langchain_ollama.llms import OllamaLLM 
llm = OllamaLLM(base_url="http://192.168.200.204:11434",model="llama3.1:8b")


In [32]:
pattern_extraction_prompt_rule = """ You are a cybersecurity analysis expert. I have a dataset of security samples. Each row is a sample containing the following fields:
- Time period (start time and end time)
- Event type
- Host
- Class label of this event (such as "false positive")
- Counts of this event during this period

Your task is to analyze and summarize the patterns and dependencies based on the following two aspects:

1. **Analyze the following sequence of events. Identify any patterns or recurring behaviors in this chunk that could lead to a potential outcome or alert. \
    Describe the conditions under which certain events lead to specific outcomes.

2. **From the following chunk of events, extract potential causal relationships between event types and their outcomes. \
    Summarize the conditions under which an event seems to lead to a specific action or state.



### Example Data (time period, event type, host, class label) are as follows: {sum_ip}


**Instructions**:
- Format the analysis as a **numbered list**, highlighting significant findings in each of the five aspects.
            """

In [33]:
 prompt_rule = """ You are a cybersecurity analysis expert. I have a dataset of security samples. Each row is a sample containing the following fields:
- Time period (start time and end time)
- Event type
- Host
- Class label of this event (such as "false positive")
- Counts of this event during this period

Your task is to analyze and summarize the patterns and dependencies based on the following five aspects:

1. **Event Sequence Dependency**:
   - Summarize any notable patterns where certain events tend to follow other events in sequence.
   - Identify any common sequences where certain types of event occurs before others.

2. **Time Dependency**:
   - Summarize patterns in the timing of events. Do certain events occur more frequently during specific times of the day or specific time periods?
   - Identify if there are any time-based patterns, such as events that occur during the night or specific hours.

3. **Host Dependency**:
   - Summarize patterns where specific hosts are more likely to experience certain types of events.
   - Identify any hosts that are frequently associated with certain events or attacks.

4. **Event Frequency and Volume Dependency**:
   - Identify patterns in the frequency and volume of events, especially when certain events occur in high volumes or low frequencies.
   - Recognize if large volumes of events in a short period could indicate floods or DDoS attacks, or if low-frequency events suggest stealthy, persistent threats.

5. **Event Evolution and Escalation Patterns**:
   - Analyze if certain low-priority events often escalate into more serious incidents.
   - Look for patterns where an event sequence might start with a benign or low-severity event but escalate to a more severe attack.

### Example Data (time period, event type, host, class label) are as follows: {sum_ip}


**Instructions**:
- Format the analysis as a **numbered list**, highlighting significant findings in each of the five aspects.
            """
        
prompt_rule_update = """ You are a cybersecurity analysis expert. Based on the previous data and summarized rules, I now have additional samples for you to analyze. Each row is a sample containing the following fields:
- Time period (start time and end time)
- Event type
- Host
- Class label of this event (such as "false positive")
- Counts of this event during this period
                Here are the previously summarized rules: 
                {response}
                
                The summarized rules are analysed from 5 aspects.
                
                Following are the new data: 
                {sum_ip}
                

               ### Instructions:
               1. Analyze the samples according to the 5 aspects of the previous summarized rules.
               2. Integrate any new patterns or dependencies found from the new data into the corresponding category in the 5 aspects.
               3. **Ensure that your analysis follows the same numbered format** as the previous summarized rules.
               4. If the new data introduces new patterns, add new rules to the relevant aspect and clearly explain how they relate to the previously summarized rules.
               5. **Provide the final updated summary in the form of a numbered list**, with each of the 5 aspects updated accordingly. """
             

In [34]:
N = 100
batch_size= 5
batches = [X_train_sort_IP[i:i+N] for i in range(0,batch_size)]

rules =''
for i in range(batch_size):
    print(f"Reading the {i}th batch data")
    if i==0:
        context = {"sum_ip": batches[i]}
        prompt_rule_formated = prompt_rule.format(**context)
        rules = llm(prompt_rule_formated)
    else:
        context = {"response": rules, "sum_ip": batches[i]}
        prompt_rule_formated = prompt_rule_update.format(**context)
        rules = llm(prompt_rule_formated)
        
    # if i==5:
    #     break
        
print(rules)

Reading the 0th batch data


/tmp/ipykernel_1769042/1082800399.py:11: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  rules = llm(prompt_rule_formated)


Reading the 1th batch data
Reading the 2th batch data
Reading the 3th batch data
Reading the 4th batch data
Based on the provided data, I will analyze the samples according to the 5 aspects of the previous summarized rules.

**1. Event Sequence Dependency:**

The new data still shows that DNS-related events (A-Dns-Clc1) are often preceded by false_positive labels, suggesting initial reconnaissance followed by malicious activity. Additionally, A-Dns-Clc1 events are consistently followed by another A-Dns-Clc1 event with a varying number of false_positive events in between.

However, I notice that the new data introduces an additional pattern: S-Flw-Apt (false_positive) events seem to be following each other in a consistent sequence. This suggests that the attacker is becoming even more consistent in their false_positive attempts.

**Updated Analysis:** The sequence of DNS-related events (A-Dns-Clc1) still suggests initial reconnaissance followed by malicious activity, with an added patte

In [35]:
# Prediction task
prompt_predict = """
**# Prediction Task**

You are a cybersecurity expert. Each network security event is a sample with a timestamp, event description, IP address, hostname, and class label. 
Your task is to predict the class label of a sample based on its event description and the 10 preceding events.

**Examples:**

- Example 1:  
  Preceding events: {example_1}  
  Current event: {event_2}  
  Predicted Class label of current event: {label_1}
  
- Example 2:  
  Preceding events: {example_2}  
  Current event: {event_2}  
  Predicted Class label of current event: {label_2}
  
- Example 3:  
  Preceding events: {example_3}  
  Current event: {event_3}  
  Predicted Class label of current event: {label_3}
  
- Example 4:  
  Preceding events: {example_4}  
  Current event: {event_4}  
  Predicted Class label of current event: {label_4}



**Task:**  
Given the the preceding events, current event, and previous learned {rules}, predict the class label directly without algorithm: 
Preceding events: {task_precedings}  
Current event: {task_event}

Format your prediction and analysis as a numbered list:
 1. Prediction result: [predicted class label]
 2. The reason you make this prediction: 
      (1) Preceding events analysis: [ analysis the preceding events]
      (2) Current event analysis:    [ analysis the current event]
 The  predicted class label are one of the following 11 types: 'cracking','dirb','dnsteal','false_positive','network_scans',\
  'privilege_escalation','reverse_shell','service_scans','service_stop','webshell','wpscan'. 

 """

In [47]:
context = var4prompt(examples_record,n=3)
context[f"example_{4}"] = predict_withcontext.iloc[0:10,0:5]
context[f"event_{4}"]   = predict_withcontext.iloc[10][0:5]
context[f"label_{4}"]   = predict_withcontext.iloc[10][5]

context[f"task_precedings"] = predict_withcontext.iloc[1:11,0:5]
context[f"task_event"]      = predict_withcontext.iloc[11,0:5]
context['rules'] = rules

In [48]:

formated_prompt_predict = prompt_predict.format(**context)

In [50]:
response = llm(formated_prompt_predict)
print(response)

Based on the analysis provided, I predict the class label as:

**1. Prediction result: [false_positive]**

The reason I make this prediction is:

**2. The reason you make this prediction:**

**(1) Preceding events analysis:** 
The preceding events all have the same type ("W-Sys-Dov") which suggests a repetitive and likely legitimate activity, such as Dovecot authentication success. There are no indicators of malicious activity in the preceding events.

**(2) Current event analysis:**
The current event is also a "Wazuh: Dovecot Authentication Success" with the host "mail". This further reinforces the likelihood that the current event is also a false positive, given the repetitive and legitimate nature of the preceding events.

Note that there are no indicators of malicious activity in either the preceding or current events. The repetitive pattern of legitimate events suggests that the current event is likely a false positive.


1. Prediction result: **A-All-Evt**
2. The reason you make this prediction:
   (1) Preceding events analysis: 
      - The preceding events are all related to web server 400 error codes, with multiple occurrences on consecutive days.
      - Events A-Dns-Clc1 have been consistently occurring on the 45th day, suggesting a pattern of DNS-related reconnaissance.
      - The presence of false_positive labels and repeated S-Flw-Apt events indicates a sophisticated attacker's tactics.
   (2) Current event analysis: 
      - The current event is also related to web server 400 error codes, with no indication of malicious activity.
      - Considering the preceding events' patterns and the lack of malicious indicators in the current event, it's likely that this event is another occurrence of A-All-Evt (all events).